# Test NSEG dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Masking, Bidirectional
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/FVAB/dataset_de_LDS_SEEDIV.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [4]:
df[df.isnull()==True] = -10

In [5]:
def preprocessing_for_LSTM(X_train, X_test, y_train, y_test):
    X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
    y_train = to_categorical(y_train)
    X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
    y_test = to_categorical(y_test)
    return X_train, X_test, y_train, y_test

# Subject dependent

In [24]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [25]:
df

0          1          2          3          4  \
id_user session video                                                          
10      1       1      25.622467  22.612988  21.225053  19.946429  19.266408   
                2      25.496412  22.407150  20.974761  20.547296  20.101432   
                3      25.302885  22.570660  21.633718  21.230311  21.011355   
                4      25.939116  22.808869  21.332588  19.427715  18.515616   
                5      25.503216  22.350826  21.157540  19.464702  18.714055   
...                          ...        ...        ...        ...        ...   
9       3       20     25.031688  22.242642  21.493354  20.546762  20.240579   
                21     25.676278  22.638414  21.002611  20.324261  20.132503   
                22     25.323145  22.412124  21.821257  20.533943  20.393856   
                23     24.879851  22.047250  21.202716  20.121354  19.557465   
                24     25.481525  22.404174  21.688045  20.494014  20.080786   

                               5          6          7          8          9  \
id_user session video                                                          
10      1       1      23.396372  20.964809  20.035117  17.815367  16.887869   
                2      23.023210  21.083666  19.698849  17.761071  16.874142   
                3      23.156935  21.018987  20.335182  18.201790  17.557549   
                4      22.954357  20.906463  20.060412  17.828035  16.748080   
                5      23.024172  20.880395  20.269262  17.854670  16.799909   
...                          ...        ...        ...        ...        ...   
9       3       20     21.793385  18.860982  18.035320  16.896775  16.196043   
                21     21.736976  18.712277  17.463331  16.739365  16.433142   
                22     21.565260  18.816661  18.326217  17.525892  17.510356   
                23     21.748668  18.707098  17.733498  16.605061  15.808999   
                24     21.794618  18.929078  18.257980  16.976043  16.482274   

                       ...  1590  1591  1592  1593  1594  1595  1596  1597  \
id_user session video  ...                                                   
10      1       1      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                2      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                3      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                4      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                5      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
9       3       20     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                21     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                22     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                23     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                24     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   

                       1598  1599  
id_user session video              
10      1       1     -10.0 -10.0  
                2     -10.0 -10.0  
                3     -10.0 -10.0  
                4     -10.0 -10.0  
                5     -10.0 -10.0  
...                     ...   ...  
9       3       20    -10.0 -10.0  
                21    -10.0 -10.0  
                22    -10.0 -10.0  
                23    -10.0 -10.0  
                24    -10.0 -10.0  

[1080 rows x 1601 columns]

In [26]:
accuracy_test = []
model = Sequential()
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    X_trainSD, X_testSD, y_trainSD, y_testSD = preprocessing_for_LSTM(X_trainSD, X_testSD, y_trainSD, y_testSD)
    del model
    model = Sequential()
    #model.add(Masking(mask_value=-10,input_shape=(X_trainSD.shape[1], 1)))
    model.add(Bidirectional(GRU(units=128, input_shape=(X_trainSD.shape[1], 1))))
    model.add(Dense(units=4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_trainSD, y_trainSD, epochs=300, batch_size=64)
    loss, accuracy = model.evaluate(X_testSD, y_testSD)
    accuracy_test.append(accuracy)

Epoch 1/300
4/4 [==============================] - 5s 138ms/step - loss: 1.4256 - accuracy: 0.3333
Epoch 2/300
4/4 [==============================] - 0s 96ms/step - loss: 1.3525 - accuracy: 0.3917
Epoch 3/300
4/4 [==============================] - 0s 95ms/step - loss: 1.2924 - accuracy: 0.3083
Epoch 4/300
4/4 [==============================] - 0s 97ms/step - loss: 1.2827 - accuracy: 0.3333
Epoch 5/300
4/4 [==============================] - 0s 96ms/step - loss: 1.2841 - accuracy: 0.3750
Epoch 6/300
4/4 [==============================] - 0s 97ms/step - loss: 1.2659 - accuracy: 0.3708
Epoch 7/300
4/4 [==============================] - 0s 97ms/step - loss: 1.2719 - accuracy: 0.3042
Epoch 8/300
4/4 [==============================] - 0s 96ms/step - loss: 1.2595 - accuracy: 0.3750
Epoch 9/300
4/4 [==============================] - 0s 98ms/step - loss: 1.2632 - accuracy: 0.3750
Epoch 10/300
4/4 [==============================] - 0s 105ms/step - loss: 1.2558 - accuracy: 0.3750
Epoch 11/300
4/4 

In [27]:
print("ACCURACY: ", np.array(accuracy_test).mean())

ACCURACY:  0.3583333392937978


# Subject-Biased Experiment


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=44)
X_train, X_test, y_train, y_test = preprocessing_for_LSTM(X_train, X_test, y_train, y_test)

In [55]:
del model
model = Sequential()
#model.add(Masking(mask_value=-10,input_shape=(X_train.shape[1], 1)))
model.add(Bidirectional(GRU(units=32, input_shape=(X_train.shape[1], 1))))
model.add(Dense(units=4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=16)

Epoch 1/200


KeyboardInterrupt: ignored

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)

7/7 [==============================] - 3s 55ms/step - loss: 1.2250 - accuracy: 0.4167


# Subject Indipendent

In [6]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

In [7]:
X1, y1, groups1 = subject_independent_split(df,1)
X2, y2, groups2 = subject_independent_split(df,2)
X3, y3, groups3 = subject_independent_split(df,3)

In [8]:
train = [X1, X2, X3]
targets = [y1, y2, y3]
groups = [groups1, groups2, groups3]

In [9]:
logo = LeaveOneGroupOut()

In [ ]:
accuracy_all = []
for n in range (0,3):
  X = train[n]
  y = targets[n]
  group = groups[n]
  for i, (train_index,test_index) in enumerate(logo.split(X,y,group)):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    X_train, X_test, y_train , y_test = preprocessing_for_LSTM(X_train,X_test,y_train,y_test)

    model = Sequential()
    #model.add(Masking(mask_value=-10,input_shape=(X_train.shape[1], 1)))
    model.add(Bidirectional(GRU(units=32, input_shape=(X_train.shape[1], 1))))
    model.add(Dense(units=4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=200, batch_size=64)
    loss, accuracy = model.evaluate(X_test, y_test)
    accuracy_all.append(accuracy)

Epoch 1/200
6/6 [==============================] - 14s 2s/step - loss: 1.6508 - accuracy: 0.1964
Epoch 2/200
6/6 [==============================] - 7s 1s/step - loss: 1.4588 - accuracy: 0.3333
Epoch 3/200
2/6 [=========>....................] - ETA: 8s - loss: 1.4229 - accuracy: 0.2812

# TO DO: da implementare kfold
https://stackoverflow.com/questions/48085182/cross-validation-in-keras


"""
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.models import * 
from tensorflow.keras.layers import * 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def buildmodel():
    model= Sequential([
        Dense(10, activation="relu"),
        Dense(5, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return(model)

estimator= KerasRegressor(build_fn=buildmodel, epochs=100, batch_size=10, verbose=0)
kfold= RepeatedKFold(n_splits=5, n_repeats=100)
results= cross_val_score(estimator, x, y, cv=kfold, n_jobs=2)  # 2 cpus
results.mean()  # Mean MSE
"""